设置工作目录

In [1]:
train_folder = 'train'
test_folder = 'test'
my_train_folder='my_train'
my_validation_folder = 'my_validation'

建立训练集、验证集目录

In [2]:
num_classes=10
#classes=["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]
import os
if os.path.exists("my_validation") is False:
    os.mkdir("my_validation")    
    for j in range(num_classes):
        path = os.path.join(my_validation_folder, 'c' + str(j))
        os.mkdir(path)
if os.path.exists("my_train") is False:
    os.mkdir("my_train")    
    for j in range(num_classes):
        path = os.path.join(my_train_folder, 'c' + str(j))
        os.mkdir(path)

统计目录下各类图片数量

In [3]:
import glob
import numpy as np 
def img_number(path,num_classes=10):
    train_num_files=[]
    for j in range(num_classes):
        train_path = os.path.join(path, 'c' + str(j), '*.jpg')
        train_files = glob.glob(train_path)
        train_num_files.append(np.size(train_files))
    train= [('class ' + str(i)+ ' : '+ str(train_num_files[i])) for i in range(10)] 
    print(train,sum(train_num_files))
    return train_num_files,sum(train_num_files)  

img_number(train_folder)    
img_number(my_train_folder)
img_number(my_validation_folder)


['class 0 : 2489', 'class 1 : 2267', 'class 2 : 2317', 'class 3 : 2346', 'class 4 : 2326', 'class 5 : 2312', 'class 6 : 2325', 'class 7 : 2002', 'class 8 : 1911', 'class 9 : 2129'] 22424
['class 0 : 2357', 'class 1 : 2132', 'class 2 : 2173', 'class 3 : 2211', 'class 4 : 2194', 'class 5 : 2179', 'class 6 : 2187', 'class 7 : 1870', 'class 8 : 1781', 'class 9 : 1995'] 21079
['class 0 : 132', 'class 1 : 135', 'class 2 : 144', 'class 3 : 135', 'class 4 : 132', 'class 5 : 133', 'class 6 : 138', 'class 7 : 132', 'class 8 : 130', 'class 9 : 134'] 1345


([132, 135, 144, 135, 132, 133, 138, 132, 130, 134], 1345)

读取driver_imgs_list文件

In [4]:
def get_driver_data():
    dr_dict = {}
    dr_set = set()
    path = os.path.join('driver_imgs_list.csv')
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')   
        dr_dict[arr[2]] = arr[0]
        dr_set.add(arr[0])
    f.close()
    return dr_dict,dr_set

建立训练集、验证集图片(随机选取两位司机)

In [6]:
driver_img,driver_id=get_driver_data()
import random
driver_id_1=(random.choice(list(driver_id)))
driver_id_2=(random.choice(list(driver_id)))
if driver_id_1==driver_id_2:
    driver_id_2=(random.choice(list(driver_id)))
print(driver_id_1,driver_id_2)    
def creative_train_val_dataset(numclasses=10):
    for j in range(num_classes):
        train_path = os.path.join(train_folder, 'c' + str(j), '*.jpg')
        print(train_path)
        train_files = glob.glob(train_path)
        for eachfile in train_files:
            v_path=os.path.join(my_validation_folder, 'c' + str(j))
            t_path=os.path.join(my_train_folder, 'c' + str(j))
            if driver_img[eachfile.split('\\')[-1].split('.')[0]+'.jpg'] in (driver_id_1,driver_id_2):
                os.system ("copy %s %s" % (eachfile, v_path))
            else:
                os.system ("copy %s %s" % (eachfile, t_path))
creative_train_val_dataset(numclasses=10)

Read drivers data
p041 p052
train\c0\*.jpg
train\c1\*.jpg
train\c2\*.jpg
train\c3\*.jpg
train\c4\*.jpg
train\c5\*.jpg
train\c6\*.jpg
train\c7\*.jpg
train\c8\*.jpg
train\c9\*.jpg


In [7]:
img_number(train_folder)    
img_number(my_train_folder)
img_number(my_validation_folder)

['class 0 : 2489', 'class 1 : 2267', 'class 2 : 2317', 'class 3 : 2346', 'class 4 : 2326', 'class 5 : 2312', 'class 6 : 2325', 'class 7 : 2002', 'class 8 : 1911', 'class 9 : 2129'] 22424
['class 0 : 2357', 'class 1 : 2132', 'class 2 : 2173', 'class 3 : 2211', 'class 4 : 2194', 'class 5 : 2179', 'class 6 : 2187', 'class 7 : 1870', 'class 8 : 1781', 'class 9 : 1995'] 21079
['class 0 : 132', 'class 1 : 135', 'class 2 : 144', 'class 3 : 135', 'class 4 : 132', 'class 5 : 133', 'class 6 : 138', 'class 7 : 132', 'class 8 : 130', 'class 9 : 134'] 1345


([132, 135, 144, 135, 132, 133, 138, 132, 130, 134], 1345)

图片预处理及增强

In [5]:
from keras.applications import inception_resnet_v2
from keras.preprocessing.image import ImageDataGenerator
img_height, img_width=(299,299)

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        #samplewise_center=True,
        #rescale=1./127.5,
        rotation_range = 20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True,
        #fill_mode='nearest',
        preprocessing_function=inception_resnet_v2.preprocess_input)
        #data_format="channels_last")

validation_datagen = ImageDataGenerator(#samplewise_center=True,
                                        preprocessing_function=inception_resnet_v2.preprocess_input)
                                        #data_format="channels_last")

train_generator = train_datagen.flow_from_directory(
        my_train_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        my_validation_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

C:\Users\Q\Anaconda2\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 21079 images belonging to 10 classes.
Found 1345 images belonging to 10 classes.


建立神经网络

In [6]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

base_model = InceptionResNetV2(weights='imagenet', include_top=False)

from keras.preprocessing import image
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,Dense,Dropout
from keras import backend as K
from keras import optimizers
from keras import regularizers

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)

# let's add a Dropout layer
x = Dropout(0.2)(x)

# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=672,
        epochs=64,
        validation_data=train_generator,
        validation_steps=32)

Epoch 1/64
672/672 [==============================] - 391s 582ms/step - loss: 2.1841 - categorical_accuracy: 0.2173 - val_loss: 2.5050 - val_categorical_accuracy: 0.0908
Epoch 2/64
672/672 [==============================] - 378s 562ms/step - loss: 1.9823 - categorical_accuracy: 0.3462 - val_loss: 2.6305 - val_categorical_accuracy: 0.1084
Epoch 3/64
672/672 [==============================] - 376s 559ms/step - loss: 1.8549 - categorical_accuracy: 0.4019 - val_loss: 2.5769 - val_categorical_accuracy: 0.1250
Epoch 4/64
672/672 [==============================] - 378s 562ms/step - loss: 1.7531 - categorical_accuracy: 0.4433 - val_loss: 2.7756 - val_categorical_accuracy: 0.1299
Epoch 5/64
672/672 [==============================] - 395s 587ms/step - loss: 1.6831 - categorical_accuracy: 0.4616 - val_loss: 2.9493 - val_categorical_accuracy: 0.1094
Epoch 6/64
672/672 [==============================] - 374s 557ms/step - loss: 1.6271 - categorical_accuracy: 0.4794 - val_loss: 2.9744 - val_categoric